In [25]:
import geojson
import os
import pandas as pd
import cv2
import numpy as np
from sklearn.metrics import cohen_kappa_score
from glob import glob

In [26]:
def json_load(json_dir,geofile1):
    with open(os.path.join(json_dir,geofile1)) as f:
        gj = geojson.load(f)
    features = gj['features']
    return features


def polygon_rater_data_load(json_dir,geofile1):
    features1 = json_load(json_dir,geofile1)
    feature1_points = [f for f in features1 if (f["geometry"]["type"]=="Polygon") & ("isLocked" not in f["properties"].keys())]
    feature1_coords = [cords["geometry"]["coordinates"] for cords in feature1_points]
    feature1_name = []
    feature1_class = []
    for cords in feature1_points:
        if "name" not in cords["properties"].keys():
            feature1_name.append(None)
        else:
            feature1_name.append(cords["properties"]["name"])
        if "classification" not in cords["properties"].keys():
            feature1_class.append(None)
        else:
            feature1_class.append(cords["properties"]["classification"]["name"])
    feature1_df = pd.DataFrame({"coordinates":feature1_coords,"class":feature1_class, "name":feature1_name})
    return feature1_coords, feature1_class, feature1_name, feature1_df


def point_rater_data_load(json_dir,geofile2):
    features2 = json_load(json_dir,geofile2)
    features2_points = [f for f in features2 if f["geometry"]["type"]=="Point"]
    feature2_coords = [cords["geometry"]["coordinates"] for cords in features2_points]
    feature2_class = [cords["properties"]["classification"]["name"] if "classification" in (cords["properties"].keys()) else "None" for cords in features2_points ]
    feature2_name = [cords["properties"]["name"] if "name" in (cords["properties"].keys()) else "None" for cords in features2_points]
    feature2_df = pd.DataFrame({"coordinates":feature2_coords,"class":feature2_class, "name":feature2_name})
    return feature2_coords, feature2_class, feature2_name, feature2_df

def match_main_point_rater(feature1_coords,feature1_class,feature1_name,feature2_coords,feature2_class,feature2_name):
    match = []
    i=0
    for contours, class_var, name_var in zip(feature1_coords,feature1_class,feature1_name):
        #print(contours,class_var,name_var)
        j=0
        for c, class_var1, name1 in zip(feature2_coords,feature2_class,feature2_name):
            #print(c, class_var1, name1)
            #print((int(c[0]),int(c[1])))
            dist = cv2.pointPolygonTest(np.int32(np.array(contours).round()),(int(c[0]),int(c[1])),False)
            if dist>=1:
                match.append([i,class_var,name_var,j,class_var1,name1, contours, c])
            j=j+1
        i=i+1 
    df = pd.DataFrame(match, columns=["main_rater_index","main_rater_class","main_rater_object_name","rater1_index","rater1_class","rater1_object_name","polygon_coords","point_coords"])
    return df


def compute_kappa_score(df):
    #df1= df[~((df["main_rater_class"]=='None') & (df["main_rater_class"]=='None'))]
    df1=df
    print(len(df1))
    labeler1 = df1["main_rater_class"]
    labeler2 = df1["rater1_class"]
    return cohen_kappa_score(labeler1, labeler2)

def find_missing_main_rater(feature1_coords,feature1_class,feature1_name):
    missing_main_rater_list = []
    for i in range(len(feature1_coords)):
        if i not in df["main_rater_index"].values:
            print(i,feature1_name[i], feature1_coords[i])
            missing_main_rater_list.append([i,feature1_class[i],feature1_name[i],None, None,None, feature1_coords[i],None])
    missing_main_rater = pd.DataFrame(missing_main_rater_list, columns=["main_rater_index","main_rater_class","main_rater_object_name","rater1_index","rater1_class","rater1_object_name","polygon_coords","point_coords"])
    return missing_main_rater

def find_missing_rater1(feature2_coords,feature2_class,feature2_name):
    missing_rater1_list = []
    for i in range(len(feature2_class)):
        if i not in df["rater1_index"].values:
            print(i,feature2_class[i],feature2_coords[i] )
            missing_rater1_list.append([None, None,None,i,feature2_class[i],feature2_name[i], None, feature2_coords[i]])
    missing_rater1 = pd.DataFrame(missing_rater1_list,columns=["main_rater_index","main_rater_class","main_rater_object_name","rater1_index","rater1_class","rater1_object_name","polygon_coords","point_coords"])
    return missing_rater1

def map_class(l):
    if l=="Cored":
        return "(C)"
    if l=="Diffuse":
        return "(D)"
    if l=="Mature":
        return "(M)"
    if l=="Pre":
        return "(P)"
    if l=="Ghost":
        return "(G)"
    if l=="Coarse-Grained":
        return "(CG)"
    return None
    

In [27]:
json_dir1= "/gladstone/finkbeiner/steve/work/data/npsad_data/monika/Amy_plaque_Results/interrater-tool/monika_vivek_interrater_analysis/monika"
json_dir2= "/gladstone/finkbeiner/steve/work/data/npsad_data/monika/Amy_plaque_Results/interrater-tool/monika_vivek_interrater_analysis/vivek"
json_dir3 = "/gladstone/finkbeiner/steve/work/data/npsad_data/monika/Amy_plaque_Results/interrater-tool/monika_vivek_interrater_analysis/Max_calibrated"
json_dir4 = "/gladstone/finkbeiner/steve/work/data/npsad_data/monika/Amy_plaque_Results/interrater-tool/monika_vivek_interrater_analysis/Ceren_calibrated"
#geofile1 = "94213_7_Sil_1.mrxs (1).geojson"
#geofile2 = "94213_7_Sil_1.mrxs.geojson"

In [28]:
geojsons_main_rater =  glob(os.path.join(json_dir1,"*.geojson"))
geojsons_names =  [x.split("/")[-1] for x in geojsons_main_rater]

In [29]:
geofile1 = geojsons_names[0]

In [30]:
json_dir2 = json_dir3
all_geojson_df = pd.DataFrame()
for geofile1 in geojsons_names:
    print("------------------",geofile1,"---------------------")
    feature1_coords, feature1_class, feature1_name, feature1_df = polygon_rater_data_load(json_dir1,geofile1)
    feature2_coords, feature2_class, feature2_name, feature2_df = point_rater_data_load(json_dir2,geofile1)
    df = match_main_point_rater(feature1_coords,feature1_class,feature1_name,feature2_coords,feature2_class,feature2_name)
    missing_main_rater = find_missing_main_rater(feature1_coords,feature1_class,feature1_name)
    missing_rater1 = find_missing_rater1(feature2_coords,feature2_class,feature2_name)
    df_final = pd.concat([df,missing_rater1,missing_main_rater], axis=0, ignore_index=True) 
    df_final["main_rater_class"] = np.where(df_final["main_rater_class"].isna(),"None",df_final["main_rater_class"])
    df_final["rater1_class"] = np.where(df_final["rater1_class"].isna(),"None",df_final["rater1_class"])
    df_final["main_rater_annotation"] = df_final["main_rater_class"].apply(lambda l: "Polygon"+ map_class(l) if l!='None' else "")
    df_final["rater1_annotation"] = df_final["rater1_class"].apply(lambda l: "Point"+ map_class(l) if l!='None' else "")
    df_final["geojson_file"] = geofile1
    if len(all_geojson_df)==0:
        all_geojson_df = df_final
    else:
        all_geojson_df =  pd.concat([all_geojson_df,df_final], ignore_index=True)

------------------ XE11-039_1_AmyB_1.mrxs.geojson ---------------------
10 20:Cored [[[58889, 144057], [58886, 144058], [58885.02, 144058.66], [58885, 144058.67], [58884, 144059], [58881, 144061], [58878, 144063], [58875, 144065], [58874, 144066], [58871, 144068], [58868, 144070], [58867, 144071], [58865.86, 144072.7], [58865, 144073], [58862, 144075], [58861, 144075.67], [58860, 144076], [58857, 144078], [58856.6, 144078.59], [58856, 144079], [58855.71, 144079.44], [58854, 144080], [58848, 144084], [58845, 144086], [58844.6, 144086.59], [58844, 144087], [58842, 144090], [58841.73, 144090.42], [58840, 144091], [58834, 144095], [58833, 144096], [58832, 144097], [58830, 144100], [58828, 144103], [58827.33, 144105], [58826, 144107], [58825.66, 144108.02], [58825, 144109], [58824.67, 144109.98], [58824, 144111], [58824, 144112], [58824, 144114], [58824, 144115], [58824, 144117], [58824, 144118], [58824, 144119], [58824, 144120], [58824, 144121], [58824, 144122], [58824, 144123], [58824, 14

In [31]:
all_geojson_df

,main_rater_index,main_rater_class,main_rater_object_name,rater1_index,rater1_class,rater1_object_name,polygon_coords,point_coords,main_rater_annotation,rater1_annotation,geojson_file
0,0,Coarse-Grained,13:Coarse-Grained,12,Coarse-Grained,6:Coarse-Grained,"[[[57865, 144201], [57857, 144202], [57850, 14...","[57905.98, 144415.97]",Polygon(CG),Point(CG),XE11-039_1_AmyB_1.mrxs.geojson
1,1,Coarse-Grained,12:Coarse-Grained,13,Coarse-Grained,9:Coarse-Grained,"[[[55739, 143692], [55725, 143698], [55724.66,...","[55744, 143828.28]",Polygon(CG),Point(CG),XE11-039_1_AmyB_1.mrxs.geojson
2,2,Coarse-Grained,15:Coarse-Grained,0,Coarse-Grained,4:Coarse-Grained,"[[[57705, 147315], [57702, 147316], [57701, 14...","[57763.58, 147461.95]",Polygon(CG),Point(CG),XE11-039_1_AmyB_1.mrxs.geojson
3,3,Coarse-Grained,14:Coarse-Grained,4,Coarse-Grained,3:Coarse-Grained,"[[[57711, 146899], [57708, 146900], [57704, 14...","[57726.39, 147042.58]",Polygon(CG),Point(CG),XE11-039_1_AmyB_1.mrxs.geojson
4,4,Coarse-Grained,17:Coarse-Grained,5,Cored,0:Cored,"[[[55464, 146813], [55459, 146814], [55457.71,...","[55483.64, 146959.36]",Polygon(CG),Point(C),XE11-039_1_AmyB_1.mrxs.geojson
...,...,...,...,...,...,...,...,...,...,...,...
308,None,None,None,76,Diffuse,86:Diffuse,None,"[41375.98, 187259.28]",,Point(D),XE18-066_1_AmyB_1.mrxs.geojson
309,None,None,None,78,Coarse-Grained,85:Coarse-Grained,None,"[41576.53, 186933.59]",,Point(CG),XE18-066_1_AmyB_1.mrxs.geojson
310,None,None,None,83,Diffuse,34:Diffuse,None,"[38004.17, 186756.7]",,Point(D),XE18-066_1_AmyB_1.mrxs.geojson
311,None,None,None,87,Coarse-Grained,83:Coarse-Grained,None,"[41202.85, 186530.77]",,Point(CG),XE18-066_1_AmyB_1.mrxs.geojson


In [32]:
all_geojson_df_nonna_class = all_geojson_df[(all_geojson_df["main_rater_class"]!="None") & (all_geojson_df["rater1_class"]!="None")]

In [33]:
compute_kappa_score(all_geojson_df_nonna_class)

187


0.6561365458060258

In [34]:
compute_kappa_score(all_geojson_df)

313


0.3034156725598882

In [24]:
ceren_match = all_geojson_df

In [32]:
all_geojson_df[["main_rater_annotation","rater1_annotation"]].to_csv("/gladstone/finkbeiner/steve/work/data/npsad_data/monika/Amy_plaque_Results/interrater-tool/monika_vivek_interrater_analysis/output/interrater_output.csv")

In [82]:
all_geojson_df.to_csv("/gladstone/finkbeiner/steve/work/data/npsad_data/monika/Amy_plaque_Results/interrater-tool/monika_vivek_interrater_analysis/output/interrater_output_full.csv")

In [65]:
monika_vivek_output = pd.read_csv("/gladstone/finkbeiner/steve/work/data/npsad_data/monika/Amy_plaque_Results/interrater-tool/monika_vivek_interrater_analysis/output/interrater_output_full.csv")

In [66]:
monika_vivek_output.columns

Index(['Unnamed: 0', 'main_rater_index', 'main_rater_class',
       'main_rater_object_name', 'rater1_index', 'rater1_class',
       'rater1_object_name', 'polygon_coords', 'point_coords',
       'main_rater_annotation', 'rater1_annotation', 'geojson_file'],
      dtype='object')

monika_vivek_output.columns = ['Unnamed: 0', 'main_rater_index', 'main_rater_class',
       'main_rater_object_name', 'rater1_index', 'rater1_class',
       'rater1_object_name', 'polygon_coords', 'rater1_point_coords',
       'main_rater_annotation', 'rater1_annotation', 'geojson_file']

In [67]:
monika_vivek_output.drop(['Unnamed: 0'],axis=1,inplace=True)

In [69]:
monika_vivek_output["Rater"]= "Vivek"

In [70]:
ceren_match.columns = ['main_rater_index', 'main_rater_class', 'main_rater_object_name',
       'rater1_index', 'rater1_class', 'rater1_object_name', 'polygon_coords',
       'rater1_point_coords', 'main_rater_annotation', 'rater1_annotation',
       'geojson_file']

In [71]:
ceren_match["Rater"]= "Ceren"

In [72]:
all_geojson_df.columns = ['main_rater_index', 'main_rater_class', 'main_rater_object_name',
       'rater1_index', 'rater1_class', 'rater1_object_name', 'polygon_coords',
       'rater1_point_coords', 'main_rater_annotation', 'rater1_annotation',
       'geojson_file']

In [73]:
all_geojson_df["Rater"] = "Max"

In [43]:
#merged1 = pd.merge(monika_vivek_output,ceren_match, on =['main_rater_index', 'main_rater_class', 'main_rater_object_name','main_rater_annotation','geojson_file'], how="outer")

/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


In [60]:
#merged2 = pd.merge(ceren_match,all_geojson_df, on =['main_rater_index', 'main_rater_class', 'main_rater_object_name','main_rater_annotation','geojson_file'], how="inner")

In [56]:
#all_geojson_df["main_rater_index"].isna().sum()

113

In [75]:
final_concat_output = pd.concat([monika_vivek_output,ceren_match,all_geojson_df],ignore_index=True)

In [81]:
monikaa_vivek_short =  monika_vivek_output[["main_rater_index","main_rater_class","main_rater_object_name","rater1_index","rater1_class","rater1_object_name","geojson_file"]]

In [83]:
ceren_match_short = ceren_match[["main_rater_index","main_rater_class","main_rater_object_name","rater1_index","rater1_class","rater1_object_name","geojson_file"]]
ceren_match_short.columns = ["main_rater_index","main_rater_class","main_rater_object_name","rater2_index","rater2_class","rater2_object_name","geojson_file"]

In [84]:
ceren_match_short

,main_rater_index,main_rater_class,main_rater_object_name,rater2_index,rater2_class,rater2_object_name,geojson_file
0,0,Coarse-Grained,13:Coarse-Grained,13,Coarse-Grained,12:Coarse-Grained,XE11-039_1_AmyB_1.mrxs.geojson
1,1,Coarse-Grained,12:Coarse-Grained,15,Coarse-Grained,14:Coarse-Grained,XE11-039_1_AmyB_1.mrxs.geojson
2,2,Coarse-Grained,15:Coarse-Grained,0,Coarse-Grained,16:Coarse-Grained,XE11-039_1_AmyB_1.mrxs.geojson
3,3,Coarse-Grained,14:Coarse-Grained,2,Coarse-Grained,23:Coarse-Grained,XE11-039_1_AmyB_1.mrxs.geojson
4,4,Coarse-Grained,17:Coarse-Grained,3,Cored,13:Cored,XE11-039_1_AmyB_1.mrxs.geojson
...,...,...,...,...,...,...,...
297,40,Coarse-Grained,44:Coarse-Grained,None,None,None,XE18-066_1_AmyB_1.mrxs.geojson
298,54,Coarse-Grained,42:Coarse-Grained,None,None,None,XE18-066_1_AmyB_1.mrxs.geojson
299,55,Diffuse,47:Diffuse,None,None,None,XE18-066_1_AmyB_1.mrxs.geojson
300,56,Coarse-Grained,43:Coarse-Grained,None,None,None,XE18-066_1_AmyB_1.mrxs.geojson


In [87]:
ceren_match_short["main_rater_index"] = ceren_match_short["main_rater_index"].astype(float)

/tmp/ipykernel_2662888/1689992118.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ceren_match_short["main_rater_index"] = ceren_match_short["main_rater_index"].astype(float)


In [88]:
ceren_match_short["rater2_index"] = ceren_match_short["rater2_index"].astype(float)

/tmp/ipykernel_2662888/2607246303.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ceren_match_short["rater2_index"] = ceren_match_short["rater2_index"].astype(float)


In [89]:
ceren_match_short.dtypes

main_rater_index          float64
main_rater_class           object
main_rater_object_name     object
rater2_index              float64
rater2_class               object
rater2_object_name         object
geojson_file               object
dtype: object

In [86]:
monikaa_vivek_short.dtypes

main_rater_index          float64
main_rater_class           object
main_rater_object_name     object
rater1_index              float64
rater1_class               object
rater1_object_name         object
geojson_file               object
dtype: object

In [91]:
monika_vivek_ceren = pd.merge(monikaa_vivek_short, ceren_match_short, on=["main_rater_index","main_rater_class","main_rater_object_name","geojson_file"],how="outer")

In [93]:
monika_vivek_ceren["main_rater_index"].isna().sum()

179

In [94]:
monika_vivek_ceren

,main_rater_index,main_rater_class,main_rater_object_name,rater1_index,rater1_class,rater1_object_name,geojson_file,rater2_index,rater2_class,rater2_object_name
0,0.0,Coarse-Grained,13:Coarse-Grained,6.0,Coarse-Grained,9:Coarse-Grained,XE11-039_1_AmyB_1.mrxs.geojson,13.0,Coarse-Grained,12:Coarse-Grained
1,1.0,Coarse-Grained,12:Coarse-Grained,7.0,Coarse-Grained,8:Coarse-Grained,XE11-039_1_AmyB_1.mrxs.geojson,15.0,Coarse-Grained,14:Coarse-Grained
2,2.0,Coarse-Grained,15:Coarse-Grained,0.0,Coarse-Grained,11:Coarse-Grained,XE11-039_1_AmyB_1.mrxs.geojson,0.0,Coarse-Grained,16:Coarse-Grained
3,3.0,Coarse-Grained,14:Coarse-Grained,2.0,Coarse-Grained,10:Coarse-Grained,XE11-039_1_AmyB_1.mrxs.geojson,2.0,Coarse-Grained,23:Coarse-Grained
4,4.0,Coarse-Grained,17:Coarse-Grained,3.0,Cored,12:Cored,XE11-039_1_AmyB_1.mrxs.geojson,3.0,Cored,13:Cored
...,...,...,...,...,...,...,...,...,...,...
375,NaN,None,None,NaN,NaN,NaN,XE07-049_1_AmyB_1.mrxs.geojson,31.0,Cored,0:Cored
376,NaN,None,None,NaN,NaN,NaN,XE07-049_1_AmyB_1.mrxs.geojson,36.0,Diffuse,48:Diffuse
377,NaN,None,None,NaN,NaN,NaN,XE07-049_1_AmyB_1.mrxs.geojson,37.0,None,36:Unlabeled
378,NaN,None,None,NaN,NaN,NaN,XE07-049_1_AmyB_1.mrxs.geojson,42.0,Cored,37:Cored


In [98]:
all_geojson_df_short = all_geojson_df[["main_rater_index","main_rater_class","main_rater_object_name","rater1_index","rater1_class","rater1_object_name","geojson_file"]]
all_geojson_df_short.columns = ["main_rater_index","main_rater_class","main_rater_object_name","rater3_index","rater3_class","rater3_object_name","geojson_file"]

In [99]:
all_geojson_df_short

,main_rater_index,main_rater_class,main_rater_object_name,rater3_index,rater3_class,rater3_object_name,geojson_file
0,0,Coarse-Grained,13:Coarse-Grained,12,Coarse-Grained,6:Coarse-Grained,XE11-039_1_AmyB_1.mrxs.geojson
1,1,Coarse-Grained,12:Coarse-Grained,13,Coarse-Grained,9:Coarse-Grained,XE11-039_1_AmyB_1.mrxs.geojson
2,2,Coarse-Grained,15:Coarse-Grained,0,Coarse-Grained,4:Coarse-Grained,XE11-039_1_AmyB_1.mrxs.geojson
3,3,Coarse-Grained,14:Coarse-Grained,4,Coarse-Grained,3:Coarse-Grained,XE11-039_1_AmyB_1.mrxs.geojson
4,4,Coarse-Grained,17:Coarse-Grained,5,Cored,0:Cored,XE11-039_1_AmyB_1.mrxs.geojson
...,...,...,...,...,...,...,...
308,None,None,None,76,Diffuse,86:Diffuse,XE18-066_1_AmyB_1.mrxs.geojson
309,None,None,None,78,Coarse-Grained,85:Coarse-Grained,XE18-066_1_AmyB_1.mrxs.geojson
310,None,None,None,83,Diffuse,34:Diffuse,XE18-066_1_AmyB_1.mrxs.geojson
311,None,None,None,87,Coarse-Grained,83:Coarse-Grained,XE18-066_1_AmyB_1.mrxs.geojson


In [101]:
all_geojson_df_short["main_rater_index"] = all_geojson_df_short["main_rater_index"].astype(float)
all_geojson_df_short["rater3_index"] = all_geojson_df_short["rater3_index"].astype(float)

/tmp/ipykernel_2662888/3620935037.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_geojson_df_short["main_rater_index"] = all_geojson_df_short["main_rater_index"].astype(float)
/tmp/ipykernel_2662888/3620935037.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_geojson_df_short["rater3_index"] = all_geojson_df_short["rater3_index"].astype(float)


In [102]:
all_geojson_df_short

,main_rater_index,main_rater_class,main_rater_object_name,rater3_index,rater3_class,rater3_object_name,geojson_file
0,0.0,Coarse-Grained,13:Coarse-Grained,12.0,Coarse-Grained,6:Coarse-Grained,XE11-039_1_AmyB_1.mrxs.geojson
1,1.0,Coarse-Grained,12:Coarse-Grained,13.0,Coarse-Grained,9:Coarse-Grained,XE11-039_1_AmyB_1.mrxs.geojson
2,2.0,Coarse-Grained,15:Coarse-Grained,0.0,Coarse-Grained,4:Coarse-Grained,XE11-039_1_AmyB_1.mrxs.geojson
3,3.0,Coarse-Grained,14:Coarse-Grained,4.0,Coarse-Grained,3:Coarse-Grained,XE11-039_1_AmyB_1.mrxs.geojson
4,4.0,Coarse-Grained,17:Coarse-Grained,5.0,Cored,0:Cored,XE11-039_1_AmyB_1.mrxs.geojson
...,...,...,...,...,...,...,...
308,NaN,None,None,76.0,Diffuse,86:Diffuse,XE18-066_1_AmyB_1.mrxs.geojson
309,NaN,None,None,78.0,Coarse-Grained,85:Coarse-Grained,XE18-066_1_AmyB_1.mrxs.geojson
310,NaN,None,None,83.0,Diffuse,34:Diffuse,XE18-066_1_AmyB_1.mrxs.geojson
311,NaN,None,None,87.0,Coarse-Grained,83:Coarse-Grained,XE18-066_1_AmyB_1.mrxs.geojson


In [112]:
all_geojson_df_short_nonna = all_geojson_df_short[~all_geojson_df_short["main_rater_index"].isna()]

In [117]:
all_geojson_df_short_na = all_geojson_df_short[all_geojson_df_short["main_rater_index"].isna()]

In [115]:
monika_vivek_ceren_max = pd.merge(monika_vivek_ceren, all_geojson_df_short_nonna, on=["main_rater_index","main_rater_class","main_rater_object_name","geojson_file"],how="left")

In [116]:
monika_vivek_ceren_max

,main_rater_index,main_rater_class,main_rater_object_name,rater1_index,rater1_class,rater1_object_name,geojson_file,rater2_index,rater2_class,rater2_object_name,rater3_index,rater3_class,rater3_object_name
0,0.0,Coarse-Grained,13:Coarse-Grained,6.0,Coarse-Grained,9:Coarse-Grained,XE11-039_1_AmyB_1.mrxs.geojson,13.0,Coarse-Grained,12:Coarse-Grained,12.0,Coarse-Grained,6:Coarse-Grained
1,1.0,Coarse-Grained,12:Coarse-Grained,7.0,Coarse-Grained,8:Coarse-Grained,XE11-039_1_AmyB_1.mrxs.geojson,15.0,Coarse-Grained,14:Coarse-Grained,13.0,Coarse-Grained,9:Coarse-Grained
2,2.0,Coarse-Grained,15:Coarse-Grained,0.0,Coarse-Grained,11:Coarse-Grained,XE11-039_1_AmyB_1.mrxs.geojson,0.0,Coarse-Grained,16:Coarse-Grained,0.0,Coarse-Grained,4:Coarse-Grained
3,3.0,Coarse-Grained,14:Coarse-Grained,2.0,Coarse-Grained,10:Coarse-Grained,XE11-039_1_AmyB_1.mrxs.geojson,2.0,Coarse-Grained,23:Coarse-Grained,4.0,Coarse-Grained,3:Coarse-Grained
4,4.0,Coarse-Grained,17:Coarse-Grained,3.0,Cored,12:Cored,XE11-039_1_AmyB_1.mrxs.geojson,3.0,Cored,13:Cored,5.0,Cored,0:Cored
...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,NaN,None,None,NaN,NaN,NaN,XE07-049_1_AmyB_1.mrxs.geojson,31.0,Cored,0:Cored,NaN,NaN,NaN
389,NaN,None,None,NaN,NaN,NaN,XE07-049_1_AmyB_1.mrxs.geojson,36.0,Diffuse,48:Diffuse,NaN,NaN,NaN
390,NaN,None,None,NaN,NaN,NaN,XE07-049_1_AmyB_1.mrxs.geojson,37.0,None,36:Unlabeled,NaN,NaN,NaN
391,NaN,None,None,NaN,NaN,NaN,XE07-049_1_AmyB_1.mrxs.geojson,42.0,Cored,37:Cored,NaN,NaN,NaN


In [118]:
all_geojson_df_short_na

,main_rater_index,main_rater_class,main_rater_object_name,rater3_index,rater3_class,rater3_object_name,geojson_file
24,NaN,None,None,3.0,Diffuse,11:Diffuse,XE11-039_1_AmyB_1.mrxs.geojson
25,NaN,None,None,8.0,Coarse-Grained,12:Coarse-Grained,XE11-039_1_AmyB_1.mrxs.geojson
26,NaN,None,None,14.0,Diffuse,22:Diffuse,XE11-039_1_AmyB_1.mrxs.geojson
27,NaN,None,None,17.0,Cored,23:Cored,XE11-039_1_AmyB_1.mrxs.geojson
28,NaN,None,None,26.0,Coarse-Grained,26:Coarse-Grained,XE11-039_1_AmyB_1.mrxs.geojson
...,...,...,...,...,...,...,...
308,NaN,None,None,76.0,Diffuse,86:Diffuse,XE18-066_1_AmyB_1.mrxs.geojson
309,NaN,None,None,78.0,Coarse-Grained,85:Coarse-Grained,XE18-066_1_AmyB_1.mrxs.geojson
310,NaN,None,None,83.0,Diffuse,34:Diffuse,XE18-066_1_AmyB_1.mrxs.geojson
311,NaN,None,None,87.0,Coarse-Grained,83:Coarse-Grained,XE18-066_1_AmyB_1.mrxs.geojson


In [120]:
monika_vivek_ceren_max = pd.concat([monika_vivek_ceren_max, all_geojson_df_short_na], axis=0)

In [121]:
monika_vivek_ceren_max.to_csv("/gladstone/finkbeiner/steve/work/data/npsad_data/monika/Amy_plaque_Results/interrater-tool/monika_vivek_interrater_analysis/output/all_raters.csv")